In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM



In [2]:
# load model and dataset
model = AutoModelForCausalLM.from_pretrained('qcw2333/YingLong_6m', trust_remote_code=True,torch_dtype=torch.bfloat16).cuda()
df = pd.read_csv("https://raw.githubusercontent.com/WenWeiTHU/TimeSeriesDatasets/refs/heads/main/ETT-small/ETTh2.csv")


In [3]:
# perpare input
lookback_length = 1440    
seqs = torch.tensor(df["OT"][:lookback_length]).unsqueeze(0).bfloat16().cuda()

In [4]:
# generate forecast 


In [3]:
fig, axs = plt.subplots(2, 1, sharex=True,figsize=(12, 6))


# vanilla  forecasting
prediction_length = 96    
output = model.generate(seqs, future_token=prediction_length)
output = output.float().cpu().detach().numpy()

axs[0].plot(range(lookback_length, lookback_length + prediction_length),df["OT"][lookback_length:lookback_length + prediction_length], color="limegreen", label="Groundtruth")
axs[0].plot(df["OT"][:lookback_length], color="royalblue", label="Lookback")
err=   np.mean((output[0,:,49] - df["OT"][lookback_length:lookback_length + prediction_length])**2)
err1=  np.mean((output[0,:,49] - df["OT"][lookback_length:lookback_length + prediction_length]).abs())
axs[0].plot(range(lookback_length, lookback_length + prediction_length), output[0,:,49], color="tomato", label=f"Vanilla")
axs[0].legend()
axs[0].grid()
axs[0].set_title(f'MSE: {err:.4f}, MAE: {err1:.4f}')


NameError: name 'plt' is not defined

In [ ]:
# Dcot version, # of future_token is larger than prediction_length
output = model.generate(seqs, future_token=4096)
output = output.float().cpu().detach().numpy()
output = output[:,:prediction_length,:]

axs[1].plot(range(lookback_length, lookback_length + prediction_length),df["OT"][lookback_length:lookback_length + prediction_length], color="limegreen", label="Groundtruth")
axs[1].plot(df["OT"][:lookback_length], color="royalblue", label="Lookback")
err=   np.mean((output[0,:,49] - df["OT"][lookback_length:lookback_length + prediction_length])**2)
err1=  np.mean((output[0,:,49] - df["OT"][lookback_length:lookback_length + prediction_length]).abs())
axs[1].plot(range(lookback_length, lookback_length + prediction_length), output[0,:,49], color="cyan", label=f"Dcot")

axs[1].legend()
axs[1].grid()
axs[1].set_title(f'MSE: {err:.4f}, MAE: {err1:.4f}')
plt.show()
